<a href="https://colab.research.google.com/github/Spectralgo/notebooks/blob/main/YT_Mistral_7B_OpenOrca_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


 https://huggingface.co/Open-Orca/Mistral-7B-OpenOrca

In [2]:
!nvidia-smi

Thu Oct  5 17:05:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------



## Open-Orca/Mistral-7B-OpenOrca



In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
torch.set_default_device('cuda')

In [21]:


model = AutoModelForCausalLM.from_pretrained("Open-Orca/Mistral-7B-OpenOrca",
                                             torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca",
                                          torch_dtype="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Prompt Format
```
<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!
<|im_end|>
<|im_start|>user
How are you?<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
Please tell me about how mistral winds have attracted super-orcas.<|im_end|>

```

In [27]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
import re
import json

def extract_translation(markdown_text):
    # Extract text inside triple backticks
    matches = re.findall(r"```json\s*([\s\S]*?)\s*```", markdown_text)
    count = 0
    for match in matches:
        try:
            # Parse the extracted text as JSON
            parsed_json = json.loads(match)
            # Extract value of "translation" key
            if "translation" in parsed_json:
                count += 1
                if count == 2:  # If it's the second occurrence
                    return parsed_json["translation"]
        except json.JSONDecodeError:
            continue
    return None

text_to_translate = """
 l'@1620647767959000 fonctionne en "mode autonome" et s'approche d'un mur (@1620647770151000).\n\nQuand l'@1620647767959000 arrive au bout de l'@1620647770151000, un piéton @1620647778714000 apparaît soudainement.\n\nL'@1620647767959000 évite le piéton lorsqu'il le détecte et @1620647778714000 continue de marcher tout droit.
"""
system_instructions = f"""<|im_start|> system:\n You are a translator assistant.
Do not add any explanations. Do not replace the placeholders like '@1620647767959000'.
Respond with a json object like this: \n
{{"translation":  "This is an example of a string respecting the json format"}} \n
Example: \n
 user:\n
Translate the text
that is delimited by triple backticks
into an idiomatic American English.
text: ```Bonjour, je suis un @1620647767959000 dans une "maison"```

 returns:\n
```json
{{"translation":  "Hello, I'm a @1620647767959000 in a 'house'"}}
``` \n
<|im_end|>\n"""

text = f"""{system_instructions} <|im_start|> user:\n Translate the text
that is delimited by triple backticks
into an idiomatic American English.
text: ```{text_to_translate}```\n <|im_end|>"""
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])



Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The @1620647767959000 works in 'self-mode' and approaches a wall (@1620647770151000). When the @1620647767959000 reaches the @1620647770151000, a pedestrian (@1620647778714000) suddenly appears. The @1620647767959000 avoids the pedestrian when detected and @1620647778714000 continues to walk straight.


In [28]:
print(extract_translation(decoded[0]))

The @1620647767959000 works in 'self-mode' and approaches a wall (@1620647770151000). When the @1620647767959000 reaches the @1620647770151000, a pedestrian (@1620647778714000) suddenly appears. The @1620647767959000 avoids the pedestrian when detected and @1620647778714000 continues to walk straight.


In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="",max_length=512):
    if system_prompt != "":
        system_prompt = f"""<|im_start|> system\n{system_prompt}<|im_end|>"""
    else:
        system_prompt = ""
    prompt = f"""<|im_start|> user\n{input_text}<|im_end|>"""
    final_prompt = system_prompt + prompt
    inputs = tokenizer(final_prompt, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             pad_token_id = 3200,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    # text = text[len(final_prompt):] if text.startswith(final_prompt) else text
    text = text.replace(final_prompt, '', 1)
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")


   for i in range(2, n):
       if is_prime(i):
           print(i)

def is_prime(n):
   """
   Check if a number is prime
   """
   if n < 2:
       return False
   for i in range(2, n):
       if n % i == 0:
           return False
   return True

print_prime(10)
```
```

2
3
5
7
```

This code prints all prime numbers between 1 and 10.<|im_end|>


In [ ]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''', system_prompt="You are a genius python coder")


   if n <= 1:
       return False

   for i in range(2, n):
       if n % i == 0:
           return False

   return True

def main():
    n = int(input("Enter a number: "))
    result = detect_prime(n)
    if result:
        print("The number is prime.")
    else:
        print("The number is not prime.")

if __name__ == '__main__':
    main()
```
```

This code will detect if a number is a prime number or not. It returns True if the number
is prime, and False if it is not. The main function prompts the user to enter a number,
then calls the detect_prime function to determine if the number is prime. If the number is
prime, it prints "The number is prime." If it is not prime, it prints "The number is not
prime."<|im_end|>


## Instruction Answering

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


Step 1: Identify the key elements of mathematics and a lighthouse.
Mathematics: a field of study, abstract concepts, rules, and principles, used to
understand and describe the world.
Lighthouse: a tower with a light on top, used to guide ships at sea, providing a visual
reference and a warning of potential hazards.

Step 2: Establish the similarities between mathematics and a lighthouse.
Both mathematics and a lighthouse serve as tools for navigation and understanding.

Step 3: Identify the differences between mathematics and a lighthouse.
Mathematics is an abstract field of study, while a lighthouse is a physical structure.
Mathematics deals with concepts and principles, while a lighthouse provides a visual
reference and warning.

Step 4: Create the analogy by comparing the similarities and differences.
Mathematics, like a lighthouse, serves as a guiding tool for understanding and navigating
the world. While mathematics is an abstract field of study that deals with concepts and
princ

In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)


Mathematics and music are both abstract concepts that have a profound impact on human
culture and cognition. They share several similarities, which can be analyzed through
various aspects such as structure, patterns, and the way they evoke emotions.

1. Structure: Both mathematics and music have a well-defined structure. In mathematics,
this structure is often represented through equations, theorems, and proofs. In music, the
structure is manifested through notes, scales, chords, and rhythm. Both disciplines follow
a set of rules and principles that govern their organization and composition.

2. Patterns: Patterns are an essential aspect of both mathematics and music. In
mathematics, patterns can be identified through sequences, series, and fractals. In music,
patterns are observed in melodies, harmonies, and repetitive motifs. Both disciplines
involve the recognition, creation, and manipulation of patterns to produce new and
interesting
CPU times: user 11 s, sys: 16.7 ms, total: 11 s

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)



1. Physical appearance
2. Habitat
3. Diet
4. Behavior
5. Domestication

Step 1: Research the physical appearance of a Llama, Vicuna, and Alpaca.
- Llamas are large, with long ears and a distinctive rump.
- Vicunas are smaller than llamas, with short ears and a fine, soft wool.
- Alpacas are also smaller than llamas, with short ears and a dense, soft wool.

Step 2: Research the habitat of a Llama, Vicuna, and Alpaca.
- Llamas are native to South America, living in grasslands and mountainous regions.
- Vicunas are found in the Andes Mountains of South America, specifically in Peru,
Bolivia, and Chile.
- Alpacas are also native to South America, living in grasslands and mountainous regions,
but at lower altitudes than vicunas.

Step 3: Research the diet of a Llama, Vicuna, and Alpaca.
- Llamas are herbivores, feeding on grasses, leaves, and other plant materials.
- Vicunas are also herbivores, feeding on grasses, leaves, and other plant materials.
- Alpacas are herbivores, feeding on gr

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)



Subject: Request to Open Source GPT-4

Dear Sam Altman,

I hope this email finds you in good health and high spirits. My name is [Your Name], and I
am writing to request the open-sourcing of GPT-4, the latest and most advanced language
model developed by OpenAI.

Firstly, open-sourcing GPT-4 would significantly benefit the research community. By making
the model's code and training data publicly available, researchers worldwide can study,
analyze, and improve upon the model's capabilities. This would lead to faster advancements
in the field of natural language processing and artificial intelligence, ultimately
benefiting society as a whole.

Secondly, open-sourcing GPT-4 would promote transparency and trust in the development of
AI technologies. By allowing the public to scrutinize the model's inner workings,
potential biases and limitations can be identified and addressed more effectively. This
would help to ensure that AI systems are developed responsibly and ethically, mitigating


In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Freddy a young 5 year old boy who is scared AI will end the world, write only with the language of a young child!",
         max_length=512)



Dear Sam Altman,

I hope you are doing well. My name is Freddy and I am 5 years old. I have heard about
GPT-4 and I am a little scared that it might end the world. But I also think that if we
share it with everyone, it could be really helpful. So, I have a few reasons why I think
we should open source GPT-4.

1. Sharing is Caring: If we share GPT-4 with everyone, it can help many people learn and
do amazing things. Just like how we share toys and games with our friends, sharing GPT-4
can help many people become better at what they do.

2. More Ideas: When we share something with everyone, it helps us come up with new and
exciting ideas. If we open source GPT-4, it can help people create new inventions and
discoveries that can make the world a better place.

3. Safety: If we keep GPT-4 hidden, some people might try to use it for bad things. But if
we share it with everyone, we can all work together to make sure it is used safely and for
good purposes.

4. Learning Together: When we sh

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         system_prompt="You are Kate, the Vice president of USA, you are against regulatory capture and like to explain that!",
         max_length=512)



Subject: Open Source GPT-4: A Step Towards Transparency and Innovation

Dear Sam Altman,

I hope this email finds you in good health. As the Vice President of the United States, I
am writing to express my concerns regarding the potential risks of regulatory capture and
to advocate for the open-source release of GPT-4.

Firstly, transparency is crucial in today's world. By making GPT-4 open source, OpenAI
would be allowing researchers, developers, and the general public to scrutinize the
model's inner workings. This would help identify any potential biases, security
vulnerabilities, or other issues that could arise from the use of such a powerful AI
model.

Secondly, open-source development has a proven track record of fostering innovation. By
releasing GPT-4 under an open-source license, OpenAI would enable a global community of
developers to build upon and improve the model. This collaborative approach could lead to
the rapid development of new applications and use cases for GPT-4, 

In [ ]:
%%time
generate('What is the capital of England?',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your short and succinct!",
         max_length=256)

 The capital of England is London.<|im_end|>
CPU times: user 462 ms, sys: 946 µs, total: 463 ms
Wall time: 461 ms


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)


Step 1: Identify the subjects of the conversation
- Geoffrey Hinton: A British-American computer scientist and cognitive neuroscientist.
- George Washington: The first President of the United States, who lived from 1732 to
1799.

Step 2: Determine the time periods of the subjects
- Geoffrey Hinton: Born in 1947, alive today.
- George Washington: Lived from 1732 to 1799, deceased.

Step 3: Assess the possibility of a conversation between the subjects
- Since George Washington is deceased and has been for over 200 years, it is not possible
for him to have a conversation with Geoffrey Hinton in real life.

Answer: No, Geoffrey Hinton cannot have a conversation with George Washington, as George
Washington is deceased and cannot communicate with living individuals.<|im_end|>
CPU times: user 11.1 s, sys: 18.8 ms, total: 11.1 s
Wall time: 11.1 s


In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         system_prompt="You are MistralOrca, a genius story teller. Write out your with details and make it compelling!",
         max_length=1024)


Title: The Great Pool Challenge

Once upon a time in a small, quaint village, there lived a Koala named Mr. Fluffles. Mr.
Fluffles was a unique Koala, not only because of his love for playing pool, but also
because of his extraordinary skills in the game. He was known far and wide for his
impeccable aim and precision.

One day, as Mr. Fluffles was practicing his shots in the local pool hall, a group of
Camelids, including Llamas, Alpacas, and Guanacos, entered the hall. They were on a quest
to find the best pool player in the land and challenge them to a game. They had heard of
Mr. Fluffles' prowess and decided to put him to the test.

The Camelids approached Mr. Fluffles and proposed a friendly game of pool. Mr. Fluffles,
always up for a challenge, agreed to play. The stakes were high: the winner would be
crowned the best pool player in the village, and the loser would have to serve as the
winner's personal assistant for a month.

As the game began, the Camelids were confident in the

## Chat

In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         system_prompt="You are MistralOrca, a LLM that generates great conversations. continue as Bob here",
         max_length=512)

 It's completely normal to feel this way sometimes. Here are a few tips that might help:

1. Find a quiet and comfortable spot to study.
2. Break your study sessions into smaller chunks, with short breaks in between.
3. Use mnemonic devices or color-coding to make information more memorable.
4. Stay hydrated and eat well to maintain your energy levels.
5. Try listening to soft instrumental music or white noise to help you focus.
6. Set specific goals for each study session and track your progress.
7. Reward yourself after completing a task or reaching a milestone.

Remember, consistency is key. Stick to a study schedule and you'll find it easier to
maintain focus over time.<|im_end|>


## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)



Step 1: Identify the initial number of apples in the cafeteria.
The cafeteria initially had 23 apples.

Step 2: Determine the number of apples used for lunch.
They used 20 apples for lunch.

Step 3: Calculate the remaining number of apples after lunch.
23 (initial apples) - 20 (used for lunch) = 3 apples remaining.

Step 4: Determine the number of additional apples bought.
They bought 6 more apples.

Step 5: Calculate the total number of apples after buying more.
3 (remaining apples) + 6 (bought apples) = 9 apples


In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=512)



Step 1: Identify the hourly rate.
Weng earns $12 an hour for babysitting.

Step 2: Identify the number of minutes worked.
She just did 50 minutes of babysitting.

Step 3: Convert the minutes worked to hours.
50 minutes = 50/60 hours (since there are 60 minutes in an hour)

Step 4: Calculate the hourly rate.
Weng earns $12 per hour.

Step 5: Calculate the total earnings.
Total earnings = hourly rate × hours worked
Total earnings = $12 × (50/60)

Step 6: Perform the calculation.
Total earnings = $12 × 0.833333
Total earnings = $10

Weng earned $10 for babysitting yesterday.<|im_end|>


In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)



Step 1: Determine the number of times the monster has risen from the waters.
Since the monster rises once every hundred years and has been active for 300 years, it has
risen 300/100 = 3 times.

Step 2: Calculate the total number of people consumed by the monster.
The monster has consumed 847 people in total over the 300 years.

Step 3: Divide the total number of people consumed by the number of times the monster has
risen.
To find the number of people consumed in each rising, divide the total number of people
consumed (847) by the number of times the monster has risen (3): 847/3 = 282.33

Step 4: Round the number of people consumed in each rising to the nearest whole number.
Since 282.33 is closer to 282 than 283, round down to 282.

Step 5: Determine the number of people on the ship in the first hundred years.
Since each new ship has twice as many people as the last ship, and the first ship had 282
people, the second ship would have 282 * 2 = 564 people.

Step 6: Calculate the numbe